# PyMem3DG Tutorial 1 - The Very Basic
`Cuncheng Zhu, Christopher T. Lee`

This tutorial provides a brief example for setting up a membrane simulation using PyMem3DG. The extensive documentations is hosted at https://rangamanilabucsd.github.io/Mem3DG/.

In [ ]:
import pymem3dg as dg
import pymem3dg.boilerplate as dgb
from functools import partial
from pathlib import Path

## Mesh initialization
PyMem3DG currently provide functions to generate simple meshes, as well as API to read $\texttt{.ply}$ files. For example, we could generate icosphere for closed membrane simualation and existing meshes based on parametrized ellipsoids. 

In [ ]:
icoFace, icoVertex = dg.getIcosphere(1, 3)
oblate = "../sample_meshes/oblate.ply"
prolate = "../sample_meshes/prolate.ply"

## Instantiate the $\texttt{System}$:  $\texttt{Parameters}$ and  $\texttt{Options}$ 

Specification of the struct `Parameters` is largely user-defined. `Parameters` is the structured into sub-structs. For example, the parameters for bending of `Parameters p` is `p.bending`. Please refer to the documentation for detailed explanation of various entries. Below we shows the simulation setup for a homogeneous membrane undergoing osmotic shrinkage, which is a classic solution of the *shape equation* that results in the biconcave shape similar to the red blood cell. Because of the simplicity of the setup, we only specify essential physical quantities such as the bending rigidity, stretching modulus and bulk modulus, assuming the default  and omitting the majority of the parameters. 

The trajectory of the simulations will be solely determined by the initial condition (initial geometry) and parameters. In this tutorial, we will see how initial geometry affects the results. 

In [ ]:
p = dg.Parameters()

p.bending.Kbc = 8.22e-5

p.tension.form = partial(
    dgb.preferredAreaSurfaceTensionModel, modulus=0.1, preferredArea=12.4866
)
p.osmotic.form = partial(
    dgb.preferredVolumeOsmoticPressureModel,
    preferredVolume=0.7 * 3.14 * 4 / 3,
    reservoirVolume=0,
    strength=0.02,
)

## Integration and optimization

PyMem3DG provides various evolvers to integrate the membrane dynamics or find mechanical/chemical equilibrium as the energy minimizer. Please refer to the documentation for details and guidelines. For simplicity, the forward euler method, mathematically equivalent to steepest descent method for optimization is used to integrate the three systems. The following integration is pre-runned. Uncomment `integrate()` to rerun them.

We first to integrate the oblate shape and we expect to form biconcave shape at the equilibrium. To visualize the result using polyscope, please refer to the documentation and a dedicated tutorial for visualization.


In [ ]:
geo = dg.Geometry(oblate)

outputDir = Path("output/tutorial1/biconcave")
outputDir.mkdir(parents=True, exist_ok=True)

g = dg.System(geometry=geo, parameters = p)
g.initialize()

fe = dg.Euler(system = g, characteristicTimeStep = 2, 
              totalTime = 100000, savePeriod = 10000, 
              tolerance = 1e-11, outputDirectory = str(outputDir))
# Uncomment to print intermediate statistics
fe.ifPrintToConsole = True
fe.ifOutputTrajFile = True
success = fe.integrate() 


Getting ahead, we can use Polycope (https://polyscope.run/py/) to visualize the trajectory. The detail will be provide in Tutorial 4. PyMem3DG provides a wrapper function to setup the visualization for inspecting the trajectory.

The below example is for purposes of easy illustration. We do not recommend running polyscope from within a jupyter notebook as it generates a separate window. 

In [ ]:
import pymem3dg.visual as dg_vis
import polyscope as ps
# # Uncomment the next two lines to run visualization
# dg_vis.animate(str(outputDir / "traj.nc"), meanCurvature = True)
# ps.show()

Similarly, we could have the same procedure with initial condition of prolate shape.

In [ ]:
outputDir = Path("output/tutorial1/dumbbell")
outputDir.mkdir(parents=True, exist_ok=True)

geo = dg.Geometry(prolate)

g = dg.System(geometry=geo, parameters = p)
g.initialize()

fe = dg.Euler(system = g, characteristicTimeStep = 2, 
              totalTime = 100000, savePeriod = 10000, 
              tolerance = 1e-12, outputDirectory = str(outputDir))

fe.ifPrintToConsole = True
fe.ifOutputTrajFile = True

success = fe.integrate() 

In [ ]:
# Uncomment the next two lines to run visualization
# dg_vis.animate(str(outputDir / "traj.nc"), meanCurvature = True)
# ps.show()

We can also do this on an generated icosphere. However, because of the symmetry of a icosphere, it appears that we need large osmotic pressure to kick start the deformation. To change the set up of $\texttt{System}$, it is generally safer practice to reinstantiate a new $\texttt{System}$ object. However, because of the simplicity of current setup, the rest of the system does not have complex dependency on attribute changes. Therefore we directly modify the underlying attribute $K_v$ and increase it by fivefold.

In [ ]:
##! THIS EXAMPLE CAN TAKE A FEW MINUTES
outputDir = Path("output/tutorial1/star")
outputDir.mkdir(parents=True, exist_ok=True)

import pymem3dg.util as dg_util
icoVertex = dg_util.sphericalHarmonicsPerturbation(icoVertex, 5, 6, 0.1)

p.osmotic.form = partial(
    dgb.preferredVolumeOsmoticPressureModel,
    preferredVolume=0.7 * 3.14 * 4 / 3,
    reservoirVolume=0,
    strength=0.1,
)

geo = dg.Geometry(icoFace, icoVertex)

g = dg.System(geometry=geo, parameters=p)
g.initialize()

# Note that the tolerance is set to zero to run to total time or...
fe = dg.Euler(system = g, characteristicTimeStep = 2, 
              totalTime = 100000, savePeriod = 10000, 
              tolerance = 0, outputDirectory = str(outputDir))


fe.ifPrintToConsole = True
fe.ifOutputTrajFile = True
fe.integrate()

In [ ]:
# Uncomment the next two lines to run visualization
# dg_vis.animate(str(outputDir / "traj.nc"), meanCurvature = True)
# ps.show()

Notices that the simulation is not ran until the equilibrium but up to $T = 70000$. It is interrupted intentionally due to corrupted mesh quality.

Notice that initial icosphere does not have constant mean curvature due to the existence of irregular points. The initial deformation comforms to the discretization defect. With the high curvature deformation, mesh quality deteriorates. To cope with the problem, please proceed to the next tutorial, where we introduce mesh regularization and mutation for adaptive meshing.